In [1]:
!pip install tweepy==3.10.0

  Using cached tweepy-3.10.0-py2.py3-none-any.whl (30 kB)
  Attempting uninstall: tweepy
    Found existing installation: tweepy 4.6.0
    Uninstalling tweepy-4.6.0:
      Successfully uninstalled tweepy-4.6.0


In [2]:
import tweepy
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import socket
import json

In [3]:
consumer_key='your-consumer-key'
consumer_secret='your-consumer-secret'
access_token ='your-access-token'
access_secret='your-acess-secret'

In [4]:
consumer_key='your-consumer-key'
consumer_secret='your-consumer-secret'
access_token ='your-access-token'
access_secret='your-acess-secret'


In [5]:
class TweetsListener(StreamListener):
    def __init__(self,csocket):

        self.client_socket = csocket

    def on_data(self, data):
        try:
            msg = json.loads( data )
            print("new message")
            # if tweet is longer than 140 characters
            if "extended_tweet" in msg:
                # add at the end of each tweet "t_end" 
                self.client_socket.send(str(msg['extended_tweet']['full_text']+"t_end").encode('utf-8'))         
                print(msg['extended_tweet']['full_text'])
            else:
                # add at the end of each tweet "t_end" 
                self.client_socket.send(str(msg['text']+"t_end").encode('utf-8'))
                print(msg['text'])
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
    def on_error(self, status):
        print(status)
        return True

import boto3

def decode_hashtags(hashtags):
    # extract the hashtag symbol and turn multiple hashtags into a list
    if hashtags:
        return ', '.join(list(map(lambda x: x["text"], hashtags)))
    else:
        return ""

def aws_comprehend(text):
    comprehend = boto3.client(service_name='comprehend', region_name='eu-west-1')

    # Extracting Grammer syntax
    syntax_response = comprehend.detect_sentiment(Text=text, LanguageCode="es")
    sentimiento = syntax_response["Sentiment"]


    mixed_score = syntax_response["SentimentScore"]["Mixed"]
    mixed_score = str("{:.8f}".format(float(mixed_score)))

    negative_score = syntax_response["SentimentScore"]["Negative"]
    negative_score = str("{:.8f}".format(float(negative_score)))

    neutral_score = syntax_response["SentimentScore"]["Neutral"]
    neutral_score = str("{:.8f}".format(float(neutral_score)))

    positive_score = syntax_response["SentimentScore"]["Positive"]
    positive_score = str("{:.8f}".format(float(positive_score)))
    data = {'sentimiento': sentimiento, 'mixed_score':mixed_score,'negative_score':negative_score,'neutral_score':neutral_score,'positive_score':positive_score}
    return data



class TweetsListener(StreamListener):
    def __init__(self,csocket):

        self.client_socket = csocket

    def on_data(self, data):
        tweet = json.loads(data)

        if "extended_tweet" in tweet.keys():
            # print(dict_data["created_at"],  dict_data["extended_tweet"]["full_text"])
            # TODO: rewrite it into a function to process the tweet text
            tweet_text = tweet["extended_tweet"]['full_text'].split('  ')#.replace('\n', ' ').replace('\r', ' ')
            tweet_text = ''.join(tweet_text).replace('\n',' ').replace('\r', ' ')
            hashtags = tweet["extended_tweet"]["entities"]["hashtags"]
        if "text" in tweet.keys():
            tweet_text = tweet["text"].split('  ')
            tweet_text = ''.join(tweet_text).replace('\n', ' ').replace('\r', ' ')
            hashtags = tweet["entities"]["hashtags"]
        else:
            return None
        retweet = False

        
        # special handle to retweet
        if "retweeted_status" in tweet.keys():

            if "extended_tweet" in tweet["retweeted_status"]:
                # print("this is a retweet and there is 'extended_tweet' file")
                tweet_text = tweet["retweeted_status"]["extended_tweet"]['full_text'].split('  ')
                tweet_text = ''.join(tweet_text).replace('\n', ' ').replace('\r', ' ')
                hashtags = tweet["retweeted_status"]["extended_tweet"]["entities"]["hashtags"]
            elif "text" in tweet["retweeted_status"]:
                tweet_text = tweet["retweeted_status"]["text"].split('  ')
                tweet_text = ''.join(tweet_text).replace('\n', ' ').replace('\r', ' ')
                hashtags = tweet["retweeted_status"]["entities"]["hashtags"]
            else:
                return None

            retweet = True   
        
        if "user" in tweet.keys():
            if "location" in tweet['user']:
                if not None == tweet['user']['location']:
                    user_location = tweet["user"]["location"].split('  ')
                    user_location = ''.join(user_location).replace('\n', ' ').replace('\r', ' ')
                else:
                    user_location = ""
                    
        if "user" in tweet.keys():
            if "url" in tweet['user']:
                if not None == tweet['user']['url']:
                    user_url = tweet["user"]["url"].split('  ')
                    user_url = ''.join(user_url).replace('\n', ' ').replace('\r', ' ')
                else:
                    user_url = ""     

        if "user" in tweet.keys():
            if "description" in tweet['user']:
                if not None == tweet['user']['description']:
                    user_description = tweet["user"]["description"].split('  ')
                    user_description = ''.join(user_description).replace('\n', ' ').replace('\r', ' ')
                else:
                    user_description = "" 
                    
        if "entities" in tweet.keys():
            try:
                if "user_mentions" in tweet['entities']:
                    if "screen_name" in tweet['entities']['user_mentions'][0]:
                        screen_name_user_mentions = tweet['entities']['user_mentions'][0]['screen_name'].replace('\n', ' ').replace('\r', ' ')
                    else:
                        screen_name_user_mentions = ""
                else:
                    screen_name_user_mentions = ""
            except IndexError as x:
                screen_name_user_mentions = ""
                
                
        if "entities" in tweet.keys():
            try:
                if "user_mentions" in tweet['entities']:
                    if "name" in tweet['entities']['user_mentions'][0]:
                        name_user_mentions = tweet['entities']['user_mentions'][0]['name'].replace('\n', ' ').replace('\r', ' ')
                    else:
                        name_user_mentions = ""
                else:
                    name_user_mentions = ""
            except IndexError as x:
                name_user_mentions = ""
                
                
        if "entities" in tweet.keys():
            try:
                if "user_mentions" in tweet['entities']:
                    if "id_str" in tweet['entities']['user_mentions'][0]:
                        id_user_mentions = tweet['entities']['user_mentions'][0]['id_str'].replace('\n', ' ').replace('\r', ' ')
                    else:
                        id_user_mentions = ""
                else:
                    id_user_mentions = ""
            except IndexError as x:
                id_user_mentions = ""
                
                
        if "geo" in tweet.keys():
            if not None == tweet['geo']:
                tweet_geo = tweet['geo']
            else:
                tweet_geo = ""
        else:
            tweet_geo = ""

        try:
            payload = {
                'tweet': str(tweet_text).encode('utf-8').decode('utf-8'),#.encode('utf-8'), #modificar codificacion 
                'hashtags': str(decode_hashtags(hashtags)),# cuando son campos None es mejor validar y pasar a "" x
                'tweet_created_at': str(tweet['created_at']),
                'retweet_count':tweet["retweet_count"],
                'tweet_favorite_count':tweet["favorite_count"],
                'retweet':retweet,
                'truncated':tweet["truncated"],
                'tweet_id': str(tweet['id']),
                'reply_count': (tweet['reply_count']),
                'user_id_str': tweet['user']['id_str'], #id del user
                'user_name':tweet['user']['name'].encode('utf-8').decode('utf-8'),
                'user_screen_name':tweet['user']['screen_name'],
                'user_location': str(user_location).encode('utf-8').decode('utf-8'),# cuando son campos None es mejor validar y pasar a "" x
                'user_url': str(user_url), # lo mismo de arriba
                'user_description': str(user_description).encode('utf-8').decode('utf-8'), # lo mismo de arriba
                'user_protected': tweet['user']['protected'],
                'user_verified': tweet['user']['verified'],
                'user_followers_count': tweet['user']['followers_count'],
                'user_friends_count': tweet['user']['friends_count'],
                'user_listed_count': tweet['user']['listed_count'],
                'user_favourites_count': tweet['user']['favourites_count'],
                'user_statuses_count': tweet['user']['statuses_count'],
                'user_created_at': str(tweet['user']['created_at']),
                'user_profile_use_background_image': tweet['user']['profile_use_background_image'], # bool
                'user_profile_image_url_https':str(tweet['user']['profile_image_url_https']),
                'user_default_profile_background': tweet['user']['default_profile'],# si no tiene background es True, oseaa, tiene el de defecto is bool
                'user_default_profile_image': tweet['user']['default_profile_image'],
                'screen_name_user_mentions': str(screen_name_user_mentions), #list index out range validate if not is  ""
                'name_user_mentions': str(name_user_mentions).encode('utf-8').decode('utf-8'),
                'id_user_mentions': str(id_user_mentions),
                'geo': str(tweet_geo),
                'tweet_sentiment': aws_comprehend(tweet_text)
            }
            # only put the record when message is not None
            if (payload):
                # print(payload)
                # note that payload is a list
                #print(payload)
                #Data getting deliverd to Kinesis
                self.client_socket.send(str(payload['tweet']))

            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
    def on_error(self, status):
        print(status)
        return True

In [6]:
def sendData(c_socket, keyword):
    print('start sending data from Twitter to socket')
    # authentication based on the credentials
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # start sending data from the Streaming API 
    twitter_stream = Stream(auth, TweetsListener(c_socket))
    twitter_stream.filter(track = keyword, languages=["es"])

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from textblob import TextBlob

In [ ]:
if __name__ == "__main__":
    # server (local machine) creates listening socket
    s = socket.socket()
    host = "0.0.0.0"    
    port = 5554
    s.bind((host, port))
    print('socket is ready')
    # server (local machine) listens for connections
    s.listen(4)
    print('socket is listening')
    # return the socket and the address on the other side of the connection (client side)
    c_socket, addr = s.accept()
    print("Received request from: " + str(addr))
    # select here the keyword for the tweet data
    sendData(c_socket, keyword = ['esteban-streaming-spark'])

socket is ready
socket is listening
Received request from: ('172.16.110.99', 37596)
start sending data from Twitter to socket


In [2]:
!pwd

/home/ec2-user/SageMaker/rrss-caracol-01/Twitter Stream Caracol/streaming


In [3]:
!ls

parc  server_socket.ipynb  user_socket.ipynb


In [4]:
!sudo rm -r parc

In [5]:
!mkdir parc

In [7]:
!ls

parc  server_socket.ipynb  user_socket.ipynb
